In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings

In [18]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [19]:
# Importing libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [20]:
%run features_testing.ipynb

In [21]:
# Split the data into explanatory and target variables
X = train.drop("TARGET", axis=1).values
y = train["TARGET"].values
X_test = test.values

In [22]:
#from sklearn.preprocessing import PowerTransformer
#
#pt = PowerTransformer(method='yeo-johnson')
#X_pt = pt.fit_transform(X)
#X_test_pt = pt.transform(X_test)

In [23]:
# Standardization
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)
X_test_std = sc.transform(X_test)

In [24]:
# Split the original data into the training data and the validation data
X_train, X_valid, y_train, y_valid = train_test_split(X_std, y, test_size=0.3, stratify=y, random_state=0)

In [25]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd

def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",

        "num_leaves": trial.suggest_int("num_leaves", 16, 128),  # Reduced from 256
        "max_depth": trial.suggest_int("max_depth", 3, 8),  # Reduced from -1 to 12
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),  # Narrowed range

        "feature_fraction": trial.suggest_float("feature_fraction", 0.7, 1.0),  # Narrowed
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.7, 1.0),  # Narrowed
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),  # Reduced from 10

        "min_child_samples": trial.suggest_int("min_child_samples", 20, 100),  # Narrowed
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 1.0, log=True),  # Narrowed
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 1.0, log=True),  # Narrowed

        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),  # Reduced from 3000
        "verbose": -1,
        
        # GPU parameters
        "device": "gpu" ,
        "gpu_platform_id": 0 ,
        "gpu_device_id": 0 ,
    }

    # Initialize k-fold cross-validation
    # Reduced to 3 folds for faster training
    n_splits = 3
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    auc_scores = []
    
    # Convert X_std to DataFrame to avoid feature names warning
    # Create feature names if they don't exist
    if isinstance(X_std, np.ndarray):
        feature_names = [f'feature_{i}' for i in range(X_std.shape[1])]
        x_train_df = pd.DataFrame(X_std, columns=feature_names)
    else:
        x_train_df = X_std
    
    # Perform k-fold cross-validation
    for fold, (train_idx, val_idx) in enumerate(skf.split(x_train_df, y)):
        x_train_fold = x_train_df.iloc[train_idx]
        y_train_fold = y[train_idx]
        x_val_fold = x_train_df.iloc[val_idx]
        y_val_fold = y[val_idx]
        
        model = lgb.LGBMClassifier(**params)
        
        model.fit(
            x_train_fold,
            y_train_fold,
            eval_set=[(x_val_fold, y_val_fold)],
            callbacks=[
                lgb.early_stopping(stopping_rounds=30),  # Reduced from 100
                lgb.log_evaluation(period=0)  # Silent training
            ]
        )
        
        preds = model.predict_proba(x_val_fold)[:, 1]
        fold_auc = roc_auc_score(y_val_fold, preds)
        auc_scores.append(fold_auc)
        
        print(f"Trial {trial.number} - Fold {fold + 1}/{n_splits} AUC: {fold_auc:.6f}")
    
    # Return mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)
    print(f"Trial {trial.number} - Mean AUC: {mean_auc:.6f} (±{std_auc:.6f})\n")
    
    return mean_auc


# Create and run the study
# n_jobs=1 when using GPU (GPU doesn't benefit from parallel trials)
study = optuna.create_study(
    direction="maximize",
    study_name="lgbm_optimization_kfold",
    sampler=optuna.samplers.TPESampler(seed=42),  # For reproducibility
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5)  # Prune bad trials early
)

n_trials = 20 

study.optimize(
    objective, 
    n_trials=n_trials,
    n_jobs=1,  # LightGBM GPU doesn't support parallel training
    show_progress_bar=True,
    timeout=7200  # 2 hour timeout as safety measure
)

print("\nBest Mean AUC:", study.best_value)
print("Best hyperparameters:\n", study.best_params)

# Train final model with best parameters on full training data
best_params = study.best_params.copy()
best_params.update({
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "verbose": -1,

    "device": "gpu" ,
    "gpu_platform_id": 0 ,
    "gpu_device_id": 0 ,
})

# Split data for final validation
X_train_final, X_valid_final, y_train_final, y_valid_final = train_test_split(
    X_std, y, test_size=0.3, stratify=y, random_state=0
)

lgbm = lgb.LGBMClassifier(**best_params)
lgbm.fit(X_train_final, y_train_final, eval_set=[(X_valid_final, y_valid_final)])

lgbm_train_pred = lgbm.predict_proba(X_train_final)[:, 1]
lgbm_valid_pred = lgbm.predict_proba(X_valid_final)[:, 1]

print(f"\nFinal Model Performance:")
print(f"Train Score: {roc_auc_score(y_train_final, lgbm_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid_final, lgbm_valid_pred)}")

[I 2025-11-28 10:41:57,640] A new study created in memory with name: lgbm_optimization_kfold
  0%|          | 0/20 [00:00<?, ?it/s]

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[164]	valid_0's auc: 0.746769
Trial 0 - Fold 1/3 AUC: 0.746769
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[179]	valid_0's auc: 0.751195
Trial 0 - Fold 2/3 AUC: 0.751195
Training until validation scores don't improve for 30 rounds


Best trial: 0. Best value: 0.748312:   5%|▌         | 1/20 [00:13<04:23, 13.84s/it, 13.84/7200 seconds]

Early stopping, best iteration is:
[180]	valid_0's auc: 0.746972
Trial 0 - Fold 3/3 AUC: 0.746972
Trial 0 - Mean AUC: 0.748312 (±0.002040)

[I 2025-11-28 10:42:11,484] Trial 0 finished with value: 0.7483117661702626 and parameters: {'num_leaves': 58, 'max_depth': 8, 'learning_rate': 0.05395030966670229, 'feature_fraction': 0.8795975452591109, 'bagging_fraction': 0.7468055921327309, 'bagging_freq': 1, 'min_child_samples': 24, 'lambda_l1': 0.29154431891537513, 'lambda_l2': 0.02537815508265665, 'n_estimators': 737}. Best is trial 0 with value: 0.7483117661702626.
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[173]	valid_0's auc: 0.747382
Trial 1 - Fold 1/3 AUC: 0.747382
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[257]	valid_0's auc: 0.751108
Trial 1 - Fold 2/3 AUC: 0.751108
Training until validation scores don't improve for 30 rounds


Best trial: 1. Best value: 0.748852:  10%|█         | 2/20 [00:20<02:58,  9.89s/it, 20.97/7200 seconds]

...

Trial 18 - Fold 3/3 AUC: 0.750707
Trial 18 - Mean AUC: 0.751253 (±0.001348)

[I 2025-11-28 10:46:54,362] Trial 18 finished with value: 0.7512529183278366 and parameters: {'num_leaves': 72, 'max_depth': 4, 'learning_rate': 0.043312407226116824, 'feature_fraction': 0.903362077422144, 'bagging_fraction': 0.7129602548622942, 'bagging_freq': 1, 'min_child_samples': 86, 'lambda_l1': 0.0023491614730919144, 'lambda_l2': 0.1844177110368866, 'n_estimators': 690}. Best is trial 15 with value: 0.7514736659400567.
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[600]	valid_0's auc: 0.74722
Trial 19 - Fold 1/3 AUC: 0.747220
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[601]	valid_0's auc: 0.750156
Trial 19 - Fold 2/3 AUC: 0.750156
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[599]	valid_0's auc: 0.747994


Best trial: 15. Best value: 0.751474: 100%|██████████| 20/20 [05:15<00:00, 15.78s/it, 315.69/7200 seconds]

Trial 19 - Fold 3/3 AUC: 0.747994
Trial 19 - Mean AUC: 0.748457 (±0.001242)

[I 2025-11-28 10:47:13,327] Trial 19 finished with value: 0.748456972665258 and parameters: {'num_leaves': 70, 'max_depth': 5, 'learning_rate': 0.014036581759563604, 'feature_fraction': 0.9182611788035248, 'bagging_fraction': 0.9239582443530352, 'bagging_freq': 1, 'min_child_samples': 90, 'lambda_l1': 0.0012937206175193438, 'lambda_l2': 0.07359974431492625, 'n_estimators': 601}. Best is trial 15 with value: 0.7514736659400567.

Best Mean AUC: 0.7514736659400567
Best hyperparameters:
 {'num_leaves': 45, 'max_depth': 5, 'learning_rate': 0.020241572916541595, 'feature_fraction': 0.8077006944020728, 'bagging_fraction': 0.8208435992012316, 'bagging_freq': 1, 'min_child_samples': 92, 'lambda_l1': 0.17960548177377253, 'lambda_l2': 0.6669757808743925, 'n_estimators': 860}



/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



Final Model Performance:
Train Score: 0.8174869064166779
Valid Score: 0.7516898276959596


# lightGBM cause seem like this is the best for the old model
import lightgbm as lgb
# optuna para 
hyperpara = {'num_leaves': 57, 'max_depth': 4, 'learning_rate': 0.03277842323048595, 'feature_fraction': 0.9569717768433371, 'bagging_fraction': 0.8506767379185446, 'bagging_freq': 1, 'min_child_samples': 48, 'lambda_l1': 0.17636584028657937, 'lambda_l2': 0.025244809309038312, 'n_estimators': 2486}

lgbm = lgb.LGBMClassifier(**hyperpara)
lgbm.fit(X_train, y_train, eval_set= [(X_valid, y_valid)])

lgbm_train_pred = lgbm.predict_proba(X_train)[:, 1]
lgbm_valid_pred = lgbm.predict_proba(X_valid)[:, 1]

print(f"Train Score: {roc_auc_score(y_train, lgbm_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, lgbm_valid_pred)}")

### under this is the output part


In [26]:
# Make predictions for the test data
# Change model name if needed
pred = lgbm.predict_proba(X_test_std)[:, 1]

/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [27]:
# Put the prediction into the format of submission
sample_sub['TARGET'] = pred
sample_sub

,SK_ID_CURR,TARGET
0,171202,0.027374
1,171203,0.195480
2,171204,0.112429
3,171205,0.122038
4,171206,0.141326
...,...,...
61495,232697,0.157011
61496,232698,0.046755
61497,232699,0.031213
61498,232700,0.076052


In [28]:
# Create the "output" directory if it doesn't exist
output_dir = Path.cwd() / "output"
os.makedirs(output_dir, exist_ok=True)

# Specify the new output file path
output_file = output_dir / "submission.csv"

# Save the CSV file to the "output" directory
sample_sub.to_csv(output_file, index=False)